# Introduction

This notebook looks to demo how to grab data from fbref: https://fbref.com/en/.

There are two ways to grab data from fbref. We can use:
* Method 1: Using pandas .read_html()
* Method 2: Use beautiful soup which is a package used to scrape data from the internet

With the above two methods, I will look to grab the currents seasons worth of data. Data include
* league table
* season averages (xG, xA, G, A, presses per final third etc)
* player stats 
* per match data

# Imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)

# Functions

FBref beautifulSoup functions

In [2]:
def html_table_to_list_of_rows(html_table):
    """Function used to convert html tables to pandas dataframes"""
    # fetch table rows
    table_rows = html_table.find_all('tr') # find table rows

    # grab rows of data from website into list format
    table_rows_list = []

    for table_row in table_rows:
        td = table_row.find_all('th') + table_row.find_all('td') # th header, td cell of a table
        row = [data.text for data in td]
        table_rows_list.append(row) 
    
    return table_rows_list

def fbref_list_of_rows_to_df(table_rows_list, column_level):
    """Function used to convert rows of data from fbref to a df"""
    if column_level == 1:
        table_columns_names = table_rows_list[:1][0]
        table_rows = table_rows_list[1:]
    elif column_level == 2:
        table_columns_names = table_rows_list[1:2][0]
        table_rows = table_rows_list[2:]
    else:
        raise ValueError('Can only take 1 or 2 for column level argument')
        
    # create dataframe for above data
    table_df = pd.DataFrame(data=table_rows, columns=table_columns_names)
    
    return table_df

def get_prem_league_table(season):
    # grab data from website
    page = requests.get(
        f"https://fbref.com/en/comps/9/{season}-{season+1}/{season}-{season+1}-Premier-League-Stats"
    )
    soup = BeautifulSoup(page.content, 'html.parser')

    # find table
    prem_league_table = soup.find(id=f"results{season}-{season+1}91_overall") 

    # get table data into list of lists
    prem_league_table_rows_list = html_table_to_list_of_rows(prem_league_table)
    
    # create df
    prem_league_table_df = fbref_list_of_rows_to_df(prem_league_table_rows_list, 1)

    
    return prem_league_table_df

def get_team_stats(season):
    # grab data from website
    page = requests.get(
        f"https://fbref.com/en/comps/9/{season}-{season+1}/{season}-{season+1}-Premier-League-Stats"
    )
    soup = BeautifulSoup(page.content, 'html.parser')

    # find table
    prem_league_stats_table = soup.find(id="stats_squads_standard_for") 

    # get tabel data into list of lists
    prem_league_stats_table_rows_list = html_table_to_list_of_rows(prem_league_stats_table)
    
    # create df
    prem_league_stats_df = fbref_list_of_rows_to_df(prem_league_stats_table_rows_list, 2)
    
    
    return prem_league_stats_df

def get_big5_player_stats(season):
    # grab data from website
    page = requests.get(
        f"https://fbref.com/en/comps/Big5/{season}-{season+1}/stats/players/{season}-{season+1}-Big-5-European-Leagues-Stats"
    )
    soup = BeautifulSoup(page.content, 'html.parser')

    # find table
    big5_player_table = soup.find(id="stats_standard") 

    # get tabel data into list of lists
    big5_player_table_rows_list = html_table_to_list_of_rows(big5_player_table)
    
    # create df
    big5_player_df = fbref_list_of_rows_to_df(big5_player_table_rows_list, 2)
    
    return big5_player_df

Utility functions

In [3]:
def flatten_cols(df):
    df.columns = ['_'.join(x) for x in
        df.columns.to_flat_index()]
    return df

# Config

We will to to extract data for the current premier league season

In [4]:
season = 2022
fbref_prem_id = 9

# Method 1: pandas .read_html()

### Load league data 

Here we are making a request for html tables which are changed to a list of dataframes that holds all standard team data we will need from fbref.

In [5]:
league_data = pd.read_html(f'https://fbref.com/en/comps/9/{season}-{season+1}/{season}-{season+1}-Premier-League-Stats')

### Fetching league table

In [6]:
regular_season_table = league_data[0]
regular_season_table_home_away = league_data[1]

In [7]:
# table for prem league table
regular_season_table.head(3)

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Arsenal,12,10,1,1,30,11,19,31,2.58,21.8,10.8,11.0,0.92,W W W D W,60135,"Gabriel Jesus, Martinelli - 5",Aaron Ramsdale,NaN
1,2,Manchester City,12,9,2,1,37,11,26,29,2.42,24.0,8.9,15.1,1.26,W W L W W,53340,Erling Haaland - 17,Ederson,NaN
2,3,Tottenham,13,8,2,3,26,16,10,26,2.00,21.2,14.5,6.8,0.52,W W L L W,61610,Harry Kane - 10,Hugo Lloris,NaN


In [8]:
# home and away data
regular_season_table_home_away.head(3)

Unnamed: 0_level_0 Unnamed: 1_level_0 Home                                 \
                  Rk              Squad   MP  W  D  L  GF GA  GD Pts Pts/MP   
0                  1            Arsenal    6  6  0  0  19  7  12  18    3.0   
1                  2    Manchester City    6  6  0  0  27  6  21  18    3.0   
2                  3          Tottenham    6  5  0  1  16  6  10  15    2.5   

                          Away                                                \
     xG  xGA   xGD xGD/90   MP  W  D  L  GF  GA GD Pts Pts/MP   xG  xGA  xGD   
0  15.4  4.8  10.6   1.77    6  4  1  1  11   4  7  13   2.17  6.4  6.0  0.4   
1  14.6  3.1  11.4   1.91    6  3  2  1  10   5  5  11   1.83  9.4  5.7  3.7   
2  12.2  5.6   6.7   1.11    7  3  2  2  10  10  0  11   1.57  9.0  8.9  0.1   

          
  xGD/90  
0   0.07  
1   0.61  
2   0.01

### Fetching standard team stats (e.g. xG, xA, goals and assist data )

In [9]:
squad_standard_stats = league_data[2]
squad_standard_stats_opponents = league_data[3]

In [10]:
squad_standard_stats.head(3)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
               Squad               # Pl                Age               Poss   
0            Arsenal                 23               24.7               57.7   
1        Aston Villa                 22               27.5               48.8   
2        Bournemouth                 25               26.8               38.6   

  Playing Time                    Performance                              \
            MP Starts   Min   90s         Gls Ast G-PK PK PKatt CrdY CrdR   
0           12    132  1080  12.0          29  22   28  1     1   19    0   
1           13    143  1170  13.0          11   6   10  1     1   27    1   
2           13    143  1170  13.0          12  10   12  0     0   20    0   

  Per 90 Minutes                          Expected                       \
             Gls   Ast   G+A  G-PK G+A-PK       xG  npxG   xAG npxG+xAG   
0           2.42  1.83  4.25  2.33   4.17     21.8  21.4  15.0     36.3   
1           0.85  0.46  1.31  0.77   1.23     14.9  14.2  10.4     24.6   
2           0.92  0.77  1.69  0.92   1.69      7.5   7.5   5.9     13.4   

  Per 90 Minutes                              
              xG   xAG xG+xAG  npxG npxG+xAG  
0           1.82  1.25   3.07  1.78     3.03  
1           1.15  0.80   1.95  1.09     1.89  
2           0.58  0.45   1.03  0.58     1.03

In [11]:
squad_standard_stats_opponents.head(3)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
               Squad               # Pl                Age               Poss   
0         vs Arsenal                 23               27.0               42.3   
1     vs Aston Villa                 22               26.8               51.2   
2     vs Bournemouth                 25               27.1               61.4   

  Playing Time                    Performance                              \
            MP Starts   Min   90s         Gls Ast G-PK PK PKatt CrdY CrdR   
0           12    132  1080  12.0          10   7    9  1     2   19    1   
1           13    143  1170  13.0          18   9   16  2     3   21    1   
2           13    143  1170  13.0          26  17   22  4     4   20    0   

  Per 90 Minutes                          Expected                       \
             Gls   Ast   G+A  G-PK G+A-PK       xG  npxG   xAG npxG+xAG   
0           0.83  0.58  1.42  0.75   1.33     10.8   9.2   8.0     17.2   
1           1.38  0.69  2.08  1.23   1.92     18.5  16.1  11.1     27.2   
2           2.00  1.31  3.31  1.69   3.00     21.3  18.2  14.0     32.2   

  Per 90 Minutes                              
              xG   xAG xG+xAG  npxG npxG+xAG  
0           0.90  0.67   1.57  0.77     1.44  
1           1.42  0.86   2.27  1.24     2.10  
2           1.64  1.08   2.72  1.40     2.48

### Fetching team defensive actions stats (e.g tackles, blocks and pressing data)

In [12]:
squad_defensive_actions_stats = league_data[16]
squad_defensive_actions_stats_opponents = league_data[17]

In [13]:
# squad defensive actions data
squad_defensive_actions_stats.head(3)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Tackles       \
               Squad               # Pl                90s     Tkl TklW   
0            Arsenal                 23               12.0     180  115   
1        Aston Villa                 22               13.0     247  131   
2        Bournemouth                 25               13.0     221  138   

                          Vs Dribbles                 Blocks           \
  Def 3rd Mid 3rd Att 3rd         Tkl  Att  Tkl% Past Blocks  Sh Pass   
0      85      59      36          77  139  55.4   62    112  27   85   
1     122      99      26         109  195  55.9   86    164  43  121   
2     111      90      20          84  182  46.2   98    154  60   94   

  Unnamed: 15_level_0 Unnamed: 16_level_0 Unnamed: 17_level_0  \
                  Int             Tkl+Int                 Clr   
0                  89                 269                 206   
1                 118                 365                 188   
2                  96                 317                 343   

  Unnamed: 18_level_0  
                  Err  
0                   8  
1                   3  
2                   0

In [14]:
# squad defensive actions data opponents
squad_defensive_actions_stats_opponents.head(3)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Tackles       \
               Squad               # Pl                90s     Tkl TklW   
0         vs Arsenal                 23               12.0     236  123   
1     vs Aston Villa                 22               13.0     198  124   
2     vs Bournemouth                 25               13.0     225  138   

                          Vs Dribbles                 Blocks           \
  Def 3rd Mid 3rd Att 3rd         Tkl  Att  Tkl% Past Blocks  Sh Pass   
0     140      79      17          94  204  46.1  110    147  60   87   
1     112      61      25          78  146  53.4   68    137  48   89   
2      91      89      45          99  177  55.9   78    128  21  107   

  Unnamed: 15_level_0 Unnamed: 16_level_0 Unnamed: 17_level_0  \
                  Int             Tkl+Int                 Clr   
0                 101                 337                 243   
1                 102                 300                 252   
2                  90                 315                 171   

  Unnamed: 18_level_0  
                  Err  
0                   8  
1                   3  
2                   3

### Fetching player stats

We can grab all top five plyers stats data all in one go.

NB: when trying to access player data via the following links - https://fbref.com/en/comps/9/stats/Premier-League-Stats, we are unable to get the players data on this page

In [15]:
top_5_stats_players = pd.read_html('https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats')
top_5_stats_players_df = top_5_stats_players[0]

In [16]:
top_5_stats_players_df.head(3)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player             Nation                Pos   
0                  1   Brenden Aaronson             us USA                 MF   
1                  2   Yunis Abdelhamid             ma MAR                 DF   
2                  3      Himad Abdelli             fr FRA                 MF   

  Unnamed: 4_level_0  Unnamed: 5_level_0 Unnamed: 6_level_0  \
               Squad                Comp                Age   
0       Leeds United  eng Premier League             22-012   
1              Reims          fr Ligue 1             35-036   
2             Angers          fr Ligue 1             22-351   

  Unnamed: 7_level_0 Playing Time                    Performance              \
                Born           MP Starts   Min   90s         Gls Ast G-PK PK   
0               2000           12     12  1014  11.3           1   1    1  0   
1               1987           13     13  1170  13.0           0   0    0  0   
2               1999            6      2   214   2.4           0   0    0  0   

                  Per 90 Minutes                          Expected            \
  PKatt CrdY CrdR            Gls   Ast   G+A  G-PK G+A-PK       xG npxG  xAG   
0     0    2    0           0.09  0.09  0.18  0.09   0.18      1.8  1.8  2.4   
1     0    1    0           0.00  0.00  0.00  0.00   0.00      0.6  0.6  0.1   
2     0    0    0           0.00  0.00  0.00  0.00   0.00      0.2  0.2  0.3   

           Per 90 Minutes                             Unnamed: 33_level_0  
  npxG+xAG             xG   xAG xG+xAG  npxG npxG+xAG             Matches  
0      4.2           0.16  0.21   0.37  0.16     0.37             Matches  
1      0.7           0.05  0.01   0.06  0.05     0.06             Matches  
2      0.5           0.09  0.13   0.22  0.09     0.22             Matches

Grab data for players in premier league

In [17]:
prem_players_df = (
    flatten_cols(top_5_stats_players_df)
    .rename(columns={'Unnamed: 5_level_0_Comp':'Competition'})
    .query("Competition == 'eng Premier League'")
)

In [18]:
prem_players_df.head(3)

,Unnamed: 0_level_0_Rk,Unnamed: 1_level_0_Player,Unnamed: 2_level_0_Nation,Unnamed: 3_level_0_Pos,Unnamed: 4_level_0_Squad,Competition,Unnamed: 6_level_0_Age,Unnamed: 7_level_0_Born,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,Performance_G-PK,Performance_PK,Performance_PKatt,Performance_CrdY,Performance_CrdR,Per 90 Minutes_Gls,Per 90 Minutes_Ast,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Expected_xG,Expected_npxG,Expected_xAG,Expected_npxG+xAG,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Unnamed: 33_level_0_Matches
0,1,Brenden Aaronson,us USA,MF,Leeds United,eng Premier League,22-012,2000,12,12,1014,11.3,1,1,1,0,0,2,0,0.09,0.09,0.18,0.09,0.18,1.8,1.8,2.4,4.2,0.16,0.21,0.37,0.16,0.37,Matches
11,12,Che Adams,sct SCO,FW,Southampton,eng Premier League,26-113,1996,12,10,913,10.1,3,1,3,0,0,0,0,0.30,0.10,0.39,0.30,0.39,3.2,3.2,1.4,4.6,0.32,0.14,0.45,0.32,0.45,Matches
12,13,Tyler Adams,us USA,MF,Leeds United,eng Premier League,23-262,1999,11,11,990,11.0,0,0,0,0,0,2,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.9,0.9,0.00,0.08,0.08,0.00,0.08,Matches


### Fetching player stats (per team)

In [19]:
arsenal_stats = pd.read_html(f'https://fbref.com/en/squads/18bb7c10/{season}-{season+1}/Arsenal-Stats')

arsenal_player_stats = arsenal_stats[0]

In [20]:
arsenal_player_stats.head(3)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
               Player             Nation                Pos   
0  Gabriel Dos Santos             br BRA                 DF   
1      Aaron Ramsdale            eng ENG                 GK   
2      William Saliba             fr FRA                 DF   

  Unnamed: 3_level_0 Playing Time                      Performance            \
                 Age           MP Starts     Min   90s         Gls  Ast G-PK   
0             24-319           12     12  1080.0  12.0         1.0  0.0  1.0   
1             24-173           12     12  1080.0  12.0         0.0  0.0  0.0   
2             21-224           12     12  1065.0  11.8         2.0  1.0  2.0   

                       Per 90 Minutes                          Expected       \
    PK PKatt CrdY CrdR            Gls   Ast   G+A  G-PK G+A-PK       xG npxG   
0  0.0   0.0  1.0  0.0           0.08  0.00  0.08  0.08   0.08      1.3  1.3   
1  0.0   0.0  0.0  0.0           0.00  0.00  0.00  0.00   0.00      0.0  0.0   
2  0.0   0.0  4.0  0.0           0.17  0.08  0.25  0.17   0.25      0.3  0.3   

                Per 90 Minutes                             Unnamed: 29_level_0  
   xAG npxG+xAG             xG   xAG xG+xAG  npxG npxG+xAG             Matches  
0  0.2      1.5           0.11  0.01   0.13  0.11     0.13             Matches  
1  0.0      0.0           0.00  0.00   0.00  0.00     0.00             Matches  
2  0.9      1.2           0.03  0.07   0.10  0.03     0.10             Matches

### Fetching team fixtures and results

In [21]:
arsenal_fixtures_results = arsenal_stats[1]

In [22]:
arsenal_fixtures_results.head(3)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,1.0,1.2,44.0,25286.0,Martin Ødegaard,4-3-3,Anthony Taylor,Match Report,NaN
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,2.7,0.5,50.0,60033.0,Martin Ødegaard,4-2-3-1,Darren England,Match Report,NaN
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,1.3,0.3,57.0,10423.0,Martin Ødegaard,4-2-3-1,Craig Pawson,Match Report,NaN


# Method 2: Beautiful soup
Helpful links:
Following link grabs data from basketball reference: https://stackoverflow.com/questions/57049359/how-to-get-table-headers-and-table-data-from-a-table-row-together-in-list

Checkout following link which contains example code for extracting fbref data with bs: https://github.com/socrstats/Medium_Football/blob/main/English%20Premier%20League%20Dendrogram.ipynb

--------------------------

The general method for using BeautifulSoup is to give the webpage of interest which will be read by the request page and then use BeautifulSoup to format the data to how we would like it.

### Fetching league table

In [23]:
prem_league_table_df = get_prem_league_table(season)
prem_league_table_df.head(3)

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Arsenal,12,10,1,1,30,11,+19,31,2.58,21.8,10.8,+11.0,+0.92,W W W D W,"60,135","Gabriel Jesus, Martinelli - 5",Aaron Ramsdale,
1,2,Manchester City,12,9,2,1,37,11,+26,29,2.42,24.0,8.9,+15.1,+1.26,W W L W W,"53,340",Erling Haaland - 17,Ederson,
2,3,Tottenham,13,8,2,3,26,16,+10,26,2.00,21.2,14.5,+6.8,+0.52,W W L L W,"61,610",Harry Kane - 10,Hugo Lloris,


### Fetch team stats

In [24]:
prem_team_stats = get_team_stats(season)
prem_team_stats.head(3)

,Squad,# Pl,Age,Poss,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls,Ast,G+A,G-PK,G+A-PK,xG,npxG,xAG,npxG+xAG,xG,xAG,xG+xAG,npxG,npxG+xAG
0,Arsenal,23,24.7,57.7,12,132,"1,080",12.0,29,22,28,1,1,19,0,2.42,1.83,4.25,2.33,4.17,21.8,21.4,15.0,36.3,1.82,1.25,3.07,1.78,3.03
1,Aston Villa,22,27.5,48.8,13,143,"1,170",13.0,11,6,10,1,1,27,1,0.85,0.46,1.31,0.77,1.23,14.9,14.2,10.4,24.6,1.15,0.80,1.95,1.09,1.89
2,Bournemouth,25,26.8,38.6,13,143,"1,170",13.0,12,10,12,0,0,20,0,0.92,0.77,1.69,0.92,1.69,7.5,7.5,5.9,13.4,0.58,0.45,1.03,0.58,1.03


### Fetch player stats

NB: appears not all rows are getting captured

In [25]:
big5_player_df = get_big5_player_stats(season)
big5_player_df.head(3)

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls,Ast,G+A,G-PK,G+A-PK,xG,npxG,xAG,npxG+xAG,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,1,Brenden Aaronson,us USA,MF,Leeds United,eng Premier League,22-012,2000,12,12,"1,014",11.3,1,1,1,0,0,2,0,0.09,0.09,0.18,0.09,0.18,1.8,1.8,2.4,4.2,0.16,0.21,0.37,0.16,0.37,Matches
1,2,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35-036,1987,13,13,"1,170",13.0,0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.6,0.6,0.1,0.7,0.05,0.01,0.06,0.05,0.06,Matches
2,3,Himad Abdelli,fr FRA,MF,Angers,fr Ligue 1,22-351,1999,6,2,214,2.4,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.2,0.2,0.3,0.5,0.09,0.13,0.22,0.09,0.22,Matches


# Conclusion

In this notebook we have shown two ways to extract data from the fbref. It appears that pandas .read_html() does most of the heavy lifting and can get the data we desire. The data grabbed through this method will need to get cleaned up, specifically with the column names used. BeautifulSoup offers an alternative solution as well but will require more functions to be created to extract data for each table we are interested in.

NB: unable to get player data from team specific pages but can overcome this by grabbing player data from competition page

### Next steps:
* Create clean functions to format data to desired format
* Do validation checks to see we are getting the amount and quality of data we would expect
* Use this data to start other projects of use